In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

Loading imports


In [3]:
torch.cuda.set_device(0)

In [4]:
path_appfeat512 = path/'appian_features_512'
path_appfeat512_tst = path/'appian_tst_features_512'

In [5]:
dls_feat = get_3d_dls_feat(Meta.df_comb, path=path_appfeat512, bs=32)

In [6]:
dls_feat.one_batch()[0].shape

torch.Size([32, 60, 32768])

## Model

In [9]:
class NeuralNet(nn.Module):
    def __init__(self, n_classes=6, embed_size=1024*4, LSTM_UNITS=1024*4, DO = 0.3):
        super(NeuralNet, self).__init__()
        
#         self.embedding_dropout = SpatialDropout(0.0) #DO)
        self.flat = nn.Sequential(AdaptiveConcatPool2d(), Flatten())
        self.hook = ReshapeBodyHook(self.flat)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)
        self.linear2 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)

        self.linear = nn.Linear(LSTM_UNITS*2, n_classes)

    def forward(self, x):
        h_embedding = self.flat(x.view(*x.shape[:2], -1, 4, 4))
        h_embadd = torch.cat((h_embedding, h_embedding), -1)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        h_conc_linear1  = F.relu(self.linear1(h_lstm1))
        h_conc_linear2  = F.relu(self.linear2(h_lstm2))
        
#         print([x.shape for x in [h_lstm1, h_lstm2, h_conc_linear1, h_conc_linear2, h_embadd]])
        
        hidden = h_lstm1 + h_lstm2 + h_conc_linear1 + h_conc_linear2 + h_embadd

        output = self.linear(hidden)
        
        return output

In [10]:
m = NeuralNet()
name = 'train3d_appian_feat_lstm_2ndplace'
learn = get_learner(dls_feat, m, name=name)
learn.add_cb(DePadLoss())

In [12]:
# learn.summary()

## Training

In [ ]:
learn.lr_find()

In [16]:
do_fit(learn, 10, 1e-3)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.055647,0.057321,0.981282,0.963460,06:13
1,0.056031,0.054906,0.981934,0.967076,05:42
2,0.056923,0.055670,0.981232,0.967662,05:47
3,0.054590,0.054923,0.981711,0.966822,05:48
4,0.050172,0.054087,0.982401,0.966096,05:51
5,0.050415,0.052792,0.982668,0.967280,05:52
6,0.047601,0.052917,0.982454,0.965828,05:40
7,0.043977,0.052119,0.982679,0.967051,05:40
8,0.043837,0.050415,0.983364,0.968528,05:40


RuntimeError: CUDA out of memory. Tried to allocate 1.00 GiB (GPU 0; 23.65 GiB total capacity; 19.32 GiB already allocated; 131.00 MiB free; 2.59 GiB cached)

In [17]:
learn.save(f'runs/{name}-1')

## Testing

In [11]:
learn.dls = get_3d_dls_feat(Meta.df_tst, path=path_appfeat512_tst, bs=32, test=True)

In [12]:
sub_fn = f'subm/{name}'
learn.load(f'runs/{name}-1')

In [13]:
preds,targs = learn.get_preds()
preds.shape, preds.min(), preds.max()

(torch.Size([121232, 6]), tensor(2.4032e-14), tensor(1.0000))

In [14]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [15]:
api.competition_submit(f'{sub_fn}.csv', name, 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:06<00:00, 4.44MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [21]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14381666,
 'totalBytes': 27277209,
 'date': '2020-02-04T23:16:51.077Z',
 'description': 'train3d_appian_feat_lstm_2ndplace',
 'errorDescription': None,
 'fileName': 'train3d_appian_feat_lstm_2ndplace.csv',
 'publicScore': '0.47366',
 'privateScore': '0.04859',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14381666/14381666.raw'}